N. Tarr 7/19/2018

Python 2.7

Description: Match SGCN species to GAP species.  Begin by matching on TSN codes.  Running this code requires access to the gapproduction package, which requires arcpy and connection to the NCSU GAP databases.  

Note: It is necessary to have access to info from NCSU GAP databases on gap species name-ITIS crosswalk info.  Here, it is accessed with a csv file.  Each GAP species is linked with an ITIS TSN.  However, some GAP species are not recognized by ITIS or direct matches will require extensive review of literature.  GAP species may be linked to a "parent" TSN (e.g. for the Genus) in such cases.

In [142]:
import pandas as pd
import sys
import pprint
execfile("T:/Scripts/AppendPaths27.py")
execfile("T:/Scripts/AppendGAPAnalysis.py")
import gapproduction as gp
import pymongo

Functions and variables for connecting to bis database

In [143]:
host="54.91.95.139"
port=27017
username=raw_input("username: ")
password=raw_input("password: ")

def _connect_mongodb(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = pymongo.MongoClient(mongo_uri)
    else:
        conn = pymongo.MongoClient(host, port)
        
    return conn[db]

def get_collection_cursor(db, collection, host, port, username, password, query={}):
    """ Read from Mongo """

    # Connect to MongoDB
    db = _connect_mongodb(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)
    
    return cursor

username: ntarr
password: LrPS7Q_gq!mw]kz


## Create a dictionary to hold SGCN - GAP matches

In [144]:
matches = {}
non_matches = {}

## Read in GAP Scientific Names, TSN codes, and species codes

In [145]:
# Read in csv of gap species, species codes, and TSN codes and notes
gapSpecies = pd.read_csv("T:/SALCC/GAPspecies2.csv")
gapSpecies.drop(["scientific_name", "common_name", "strComName"],inplace=True, axis=1)
gapSpecies.drop(columns=["Unnamed: 0"],inplace=True, axis=1)
gapSpecies['intITIScode'].fillna(0, inplace=True)
gapSpecies['intITIScode'] = [int(i) for i in gapSpecies['intITIScode']]

# Print first 5 records
gapSpecies.head()

,strUC,strSciName,intITIScode,intGapITISmatch,strGapITISmatch,memTaxaNotes
0,rLWWHx,Aspidoscelis gypsi,174021,6.0,ITIS does not recognize this species concept. ...,Aspidoscelis gypsi (Little White Whiptail) is ...
1,rDENIx,Hypsiglena chlorophaea,174233,6.0,ITIS does not recognize this species concept. ...,Hypsiglena chlorophaea (Desert Nightsnake) is ...
2,rCHNIx,Hypsiglena jani,174233,6.0,ITIS does not recognize this species concept. ...,Hypsiglena jani (Chihuahuan Nightsnake) is rec...
3,rWNLIx,Xantusia wigginsi,174092,6.0,ITIS does not recognize this species concept. ...,"Based on mitochondrial and nuclear DNA data, S..."
4,mPYRAc,Brachylagus idahoensis pop. 2,552521,5.0,ITIS does not recognize this sub-population co...,NatureServe recognizes a distinct subpopulatio...


## Read in SGCN Scientific Names and TSN codes
Define the search query for filtering out unwanted records from the bis database.  No need for aquatic or invertebrates or species only listed in AK.

In [146]:
# Get a list of state names for the query.  We will want to exclude non-lower 48 state species.
lower48s = list(set(gp.dictionaries.stateDict_To_Abbr.keys()) - set(["Northern Mariana Islands", "Alaska", "Puerto Rico", 
                                                           "American Samoa", "Guam", "Virgin Islands"]))
print(lower48s)

# Get a list of taxanomic groups GAP modeled (terrestrial)
terrSpecies = ["Birds", "Reptiles", "Amphibians", "Mammals"]

query = {"State Summary.2015.State List": { "$in": lower48s }, 
         "State Summary.2005.State List": { "$in": lower48s },
         "Taxonomic Group": {"$in": terrSpecies}
         }

['Mississippi', 'Iowa', 'Oklahoma', 'Wyoming', 'Minnesota', 'Illinois', 'Georgia', 'Arkansas', 'New Mexico', 'Indiana', 'Maryland', 'Louisiana', 'Texas', 'Arizona', 'Wisconsin', 'Michigan', 'Kansas', 'Utah', 'Virginia', 'Oregon', 'Connecticut', 'New York', 'California', 'Idaho', 'West Virginia', 'South Carolina', 'New Hampshire', 'Massachusetts', 'Vermont', 'Delaware', 'North Dakota', 'Pennsylvania', 'Florida', 'Hawaii', 'Kentucky', 'Rhode Island', 'Nebraska', 'Missouri', 'Ohio', 'Alabama', 'South Dakota', 'Colorado', 'New Jersey', 'American Somoa', 'Washington', 'North Carolina', 'Tennessee', 'Montana', 'District of Columbia', 'Nevada', 'Maine']


## Attempt to match with GAP

### One-to-one TSN matches
Process each document in the collection. for each document/record, check to see if the SGCN TSN is associated with a GAP species.  If it is, collect that record and match.  Note that there are some one-to-many matches on tsn (one SGCN TSN to many GAP species; sometimes due to subspecies).  Those aren't resolved here; they get collected as non-matches.

# Note!! are if else statements correct?

In [147]:
# Get a cursor for the SGCN Synthesis collection from bis
SGCNcursor = get_collection_cursor(db='bis', collection='SGCN Synthesis', query=query, host=host, port=port, 
                                   username=username, password=password)

# Loop through SGCN's and collect matches on TSN
for x in SGCNcursor:
    tsn = int(x['Taxonomic Authority ID'][-6:])
    sciname = x['Taxonomy'][-1]['name']
    if int(tsn) in set(gapSpecies.intITIScode):
        GAPtsnDF = gapSpecies[gapSpecies['intITIScode'] == tsn]
        if len(GAPtsnDF) == 1:
            spInfo = {}
            spInfo["SGCN_id"] = sciname
            spInfo["SGCN-ITIS_TSN"] = tsn
            spInfo["SGCN_common_name"] = x['Common Name']
            spInfo["GAP-ITIS_TSN"] = tsn
            spInfo["GAP_scientific_name"] = GAPtsnDF["strSciName"].unique()[0]
            spInfo["GAP_common_name"] = gp.gapdb.NameCommon(GAPtsnDF["strUC"].unique()[0])
            spInfo["GAP_species_code"] = GAPtsnDF["strUC"].unique()[0]
            spInfo["GAP-ITIS_match_notes"] = GAPtsnDF["strGapITISmatch"].unique()[0]
            spInfo["GAP_taxa_notes"] = GAPtsnDF["memTaxaNotes"].unique()[0]
            spInfo["SGCN-GAP_match_basis"] = "TSN" 
            spInfo["SGCN-GAP_match_notes"] = "exact match"
            matches[sciname] = spInfo
    else:
        spInfo = {}
        spInfo["SGCN_scientific_name"] = sciname
        spInfo["SGCN-ITIS_TSN"] = tsn
        spInfo["SGCN_common_name"] = x['Common Name']
        non_matches[sciname] = spInfo

### Display results
Show the number of records in each dictionary

In [148]:
print("SGCN Synthesis documents: " + str(len(list(get_collection_cursor(db='bis', collection='SGCN Synthesis', 
                                                                        query=query, host=host, port=port,
                                                                        username=username, password=password)))))
print("GAP species: 1719")
print("Matches: " + str(len(matches.keys())))
print("Non-matches: " + str(len(non_matches.keys())))

SGCN Synthesis documents: 1509
GAP species: 1719
Matches: 1047
Non-matches: 443


### One-to-many TSN matches


### Display results
Show the number of records in each dictionary

In [149]:
print("SGCN Synthesis documents: " + str(len(list(get_collection_cursor(db='bis', collection='SGCN Synthesis', 
                                                                        query=query, host=host, port=port,
                                                                        username=username, password=password)))))
print("GAP species: 1719")
print("Matches: " + str(len(matches.keys())))
print("Non-matches: " + str(len(non_matches.keys())))

SGCN Synthesis documents: 1509
GAP species: 1719
Matches: 1047
Non-matches: 443


## Match on scientific names
See if any non-matches can be matched on scientific name.
# NOTE!! Giving incorrect results.

In [150]:
# Loop through non-matches and collect matches on scientific name.
for x in non_matches.keys():   
    if x in set(gapSpecies.strSciName) and x not in matches.keys():
        GAPsciDF = gapSpecies[gapSpecies['strSciName'] == x]
        spInfo = {}
        spInfo["SGCN_id"] = x
        spInfo["SGCN-ITIS_TSN"] = non_matches[x]["SGCN-ITIS_TSN"] #!!!!!
        spInfo["SGCN_common_name"] = non_matches[x]['SGCN_common_name'] #!!!!!!!
        spInfo["GAP-ITIS_TSN"] = GAPsciDF["intITIScode"].unique()[0]
        spInfo["GAP_scientific_name"] = GAPsciDF["strSciName"].unique()[0]
        spInfo["GAP_common_name"] = gp.gapdb.NameCommon(GAPsciDF["strUC"].unique()[0])
        spInfo["GAP_species_code"] = GAPsciDF["strUC"].unique()[0]
        spInfo["GAP-ITIS_match_notes"] = GAPsciDF["strGapITISmatch"].unique()[0]
        spInfo["GAP_taxa_notes"] = GAPsciDF["memTaxaNotes"].unique()[0]
        spInfo["SGCN-GAP_match_basis"] = "scientific name" 
        spInfo["SGCN-GAP_match_notes"] = "exact match"
        matches[x] = spInfo
        print("\n\t\t" + x)
        pprint.pprint(spInfo)
        del non_matches[x]
    else:
        spInfo = {}
        spInfo["SGCN_id"] = x
        spInfo["SGCN-ITIS_TSN"] = non_matches[x]["SGCN-ITIS_TSN"]
        spInfo["SGCN_common_name"] = non_matches[x]["SGCN_common_name"]
        non_matches[sciname] = spInfo

		Onychoprion fuscatus
{'GAP-ITIS_TSN': 176894,
 'GAP-ITIS_match_notes': 'ITIS valid match.',
 'GAP_common_name': u'Sooty Tern',
 'GAP_scientific_name': 'Onychoprion fuscatus',
 'GAP_species_code': 'bSOTEx',
 'GAP_taxa_notes': nan,
 'SGCN-GAP_match_basis': 'scientific name',
 'SGCN-GAP_match_notes': 'exact match',
 'SGCN-ITIS_TSN': 824105,
 'SGCN_common_name': u'Sooty Tern',
 'SGCN_id': u'Onychoprion fuscatus'}
		Onychoprion anaethetus
{'GAP-ITIS_TSN': 176897,
 'GAP-ITIS_match_notes': 'ITIS valid match.',
 'GAP_common_name': u'Bridled Tern',
 'GAP_scientific_name': 'Onychoprion anaethetus',
 'GAP_species_code': 'bBRTEx',
 'GAP_taxa_notes': nan,
 'SGCN-GAP_match_basis': 'scientific name',
 'SGCN-GAP_match_notes': 'exact match',
 'SGCN-ITIS_TSN': 824104,
 'SGCN_common_name': u'Bridled Tern',
 'SGCN_id': u'Onychoprion anaethetus'}
		Chroicocephalus philadelphia
{'GAP-ITIS_TSN': 176839,
 'GAP-ITIS_match_notes': 'ITIS valid match.',
 'GAP_common_name': u"Bonaparte's Gull",
 'GAP_scientific_

### Display results
Show the number of records in each dictionary

In [151]:
print("SGCN Synthesis documents: " + str(len(list(get_collection_cursor(db='bis', collection='SGCN Synthesis', 
                                                                        query=query, host=host, port=port,
                                                                        username=username, password=password)))))
print("GAP species: 1719")
print("Matches: " + str(len(matches.keys())))
print("Non-matches: " + str(len(non_matches.keys())))

SGCN Synthesis documents: 1509
GAP species: 1719
Matches: 1058
Non-matches: 433


# Non-matches

In [152]:
pprint.pprint(non_matches)

{u'Accipiter gentilis atricapillus': {'SGCN-ITIS_TSN': 175301,
                                      'SGCN_common_name': u'Northern Goshawk',
                                      'SGCN_scientific_name': u'Accipiter gentilis atricapillus'},
 u'Acris blanchardi': {'SGCN-ITIS_TSN': 774220,
                       'SGCN_common_name': u"Blanchard's cricket frog",
                       'SGCN_scientific_name': u'Acris blanchardi'},
 u'Acrocephalus familiaris kingi': {'SGCN-ITIS_TSN': 202291,
                                    'SGCN_common_name': u'Nihoa Millerbird',
                                    'SGCN_scientific_name': u'Acrocephalus familiaris kingi'},
 u'Agkistrodon contortrix mokasen': {'SGCN-ITIS_TSN': 174297,
                                     'SGCN_common_name': u'Northern Copperhead',
                                     'SGCN_scientific_name': u'Agkistrodon contortrix mokasen'},
 u'Agkistrodon piscivorus leucostoma': {'SGCN-ITIS_TSN': 209504,
                                

 u'Crotalus willardi obscurus': {'SGCN-ITIS_TSN': 202178,
                                 'SGCN_common_name': u'Animas Ridge-nosed Rattlesnake',
                                 'SGCN_scientific_name': u'Crotalus willardi obscurus'},
 u'Deirochelys reticularia miaria': {'SGCN-ITIS_TSN': 208628,
                                     'SGCN_common_name': u'Western Chicken Turtle',
                                     'SGCN_scientific_name': u'Deirochelys reticularia miaria'},
 u'Desmognathus': {'SGCN-ITIS_TSN': 173632,
                   'SGCN_common_name': u'Dusky Salamanders',
                   'SGCN_scientific_name': u'Desmognathus'},
 u'Diadophis punctatus acricus': {'SGCN-ITIS_TSN': 209168,
                                  'SGCN_common_name': u'Key Ringneck Snake',
                                  'SGCN_scientific_name': u'Diadophis punctatus acricus'},
 u'Diadophis punctatus arnyi': {'SGCN-ITIS_TSN': 209170,
                                'SGCN_common_name': u'Prairie Ringneck S

                                'SGCN_scientific_name': u'Leptonycteris yerbabuenae'},
 u'Lepus americanus cascadensis': {'SGCN-ITIS_TSN': 727811,
                                   'SGCN_common_name': u'Sierra Nevada snowshoe hare',
                                   'SGCN_scientific_name': u'Lepus americanus cascadensis'},
 u'Limosa haemastica': {'SGCN-ITIS_TSN': 176690,
                        'SGCN_common_name': u'Hudsonian Godwit',
                        'SGCN_scientific_name': u'Limosa haemastica'},
 u'Lithobates areolatus areolatus': {'SGCN-ITIS_TSN': 776826,
                                     'SGCN_common_name': u'Southern Crawfish Frog',
                                     'SGCN_scientific_name': u'Lithobates areolatus areolatus'},
 u'Lithobates areolatus circulosus': {'SGCN-ITIS_TSN': 776827,
                                      'SGCN_common_name': u'Northern Crawfish Frog',
                                      'SGCN_scientific_name': u'Lithobates areolatus circulosus'}

 u'Procyon lotor auspicatus': {'SGCN-ITIS_TSN': 727464,
                               'SGCN_common_name': u'Key Vaca Raccoon',
                               'SGCN_scientific_name': u'Procyon lotor auspicatus'},
 u'Procyon lotor incautus': {'SGCN-ITIS_TSN': 727472,
                             'SGCN_common_name': u'Key West Raccoon',
                             'SGCN_scientific_name': u'Procyon lotor incautus'},
 u'Progne subis arboricola': {'SGCN-ITIS_TSN': 924850,
                              'SGCN_common_name': u'Western Purple Martin',
                              'SGCN_scientific_name': u'Progne subis arboricola'},
 u'Progne subis hesperia': {'SGCN-ITIS_TSN': 178466,
                            'SGCN_common_name': u'Desert Purple Martin',
                            'SGCN_scientific_name': u'Progne subis hesperia'},
 u'Pseudemys concinna floridana': {'SGCN-ITIS_TSN': 551965,
                                   'SGCN_common_name': u'Florida cooter',
                             

 u'Thomomys bottae subsimilis': {'SGCN-ITIS_TSN': 900437,
                                 'SGCN_common_name': u'Harquahala southern pocket gopher',
                                 'SGCN_scientific_name': u'Thomomys bottae subsimilis'},
 u'Thomomys talpoides cheyennensis': {'SGCN-ITIS_TSN': 900475,
                                      'SGCN_common_name': u'Cheyenne Northern Pocket Gopher',
                                      'SGCN_scientific_name': u'Thomomys talpoides cheyennensis'},
 u'Trachemys scripta elegans': {'SGCN-ITIS_TSN': 173823,
                                'SGCN_common_name': u'Red-eared Slider',
                                'SGCN_scientific_name': u'Trachemys scripta elegans'},
 u'Trachemys scripta scripta': {'SGCN-ITIS_TSN': 173821,
                                'SGCN_common_name': u'Yellow-bellied Slider',
                                'SGCN_scientific_name': u'Trachemys scripta scripta'},
 u'Trachemys scripta troostii': {'SGCN-ITIS_TSN': 173825,
         

# Matches

In [153]:
pprint.pprint(matches)

{u'Accipiter cooperii': {'GAP-ITIS_TSN': 175309,
                         'GAP-ITIS_match_notes': 'ITIS valid match.',
                         'GAP_common_name': u"Cooper's Hawk",
                         'GAP_scientific_name': 'Accipiter cooperii',
                         'GAP_species_code': 'bCOHAx',
                         'GAP_taxa_notes': nan,
                         'SGCN-GAP_match_basis': 'TSN',
                         'SGCN-GAP_match_notes': 'exact match',
                         'SGCN-ITIS_TSN': 175309,
                         'SGCN_common_name': u"Cooper's Hawk",
                         'SGCN_id': u'Accipiter cooperii'},
 u'Accipiter gentilis': {'GAP-ITIS_TSN': 175300,
                         'GAP-ITIS_match_notes': 'ITIS valid match.',
                         'GAP_common_name': u'Northern Goshawk',
                         'GAP_scientific_name': 'Accipiter gentilis',
                         'GAP_species_code': 'bNOGOx',
                         'GAP_taxa_notes': n

                           'SGCN_id': u'Ammodramus henslowii'},
 u'Ammodramus leconteii': {'GAP-ITIS_TSN': 179345,
                           'GAP-ITIS_match_notes': 'ITIS valid match.',
                           'GAP_common_name': u"Le Conte's Sparrow",
                           'GAP_scientific_name': 'Ammodramus leconteii',
                           'GAP_species_code': 'bLCSPx',
                           'GAP_taxa_notes': nan,
                           'SGCN-GAP_match_basis': 'TSN',
                           'SGCN-GAP_match_notes': 'exact match',
                           'SGCN-ITIS_TSN': 179345,
                           'SGCN_common_name': u"Le Conte's Sparrow",
                           'SGCN_id': u'Ammodramus leconteii'},
 u'Ammodramus maritimus': {'GAP-ITIS_TSN': 179346,
                           'GAP-ITIS_match_notes': 'ITIS valid match.',
                           'GAP_common_name': u'Seaside Sparrow',
                           'GAP_scientific_name': 'Ammodramus ma

                     'GAP_scientific_name': 'Aneides hardii',
                     'GAP_species_code': 'aSMTSx',
                     'GAP_taxa_notes': nan,
                     'SGCN-GAP_match_basis': 'TSN',
                     'SGCN-GAP_match_notes': 'exact match',
                     'SGCN-ITIS_TSN': 173702,
                     'SGCN_common_name': u'Sacramento Mountains Salamander',
                     'SGCN_id': u'Aneides hardii'},
 u'Anhinga anhinga': {'GAP-ITIS_TSN': 174755,
                      'GAP-ITIS_match_notes': 'ITIS valid match.',
                      'GAP_common_name': u'Anhinga',
                      'GAP_scientific_name': 'Anhinga anhinga',
                      'GAP_species_code': 'bANHIx',
                      'GAP_taxa_notes': nan,
                      'SGCN-GAP_match_basis': 'TSN',
                      'SGCN-GAP_match_notes': 'exact match',
                      'SGCN-ITIS_TSN': 174755,
                      'SGCN_common_name': u'Anhinga',
              

                      'GAP_taxa_notes': nan,
                      'SGCN-GAP_match_basis': 'TSN',
                      'SGCN-GAP_match_notes': 'exact match',
                      'SGCN-ITIS_TSN': 174202,
                      'SGCN_common_name': u'Glossy Snake',
                      'SGCN_id': u'Arizona elegans'},
 u'Artemisiospiza belli': {'GAP-ITIS_TSN': 997724,
                           'GAP-ITIS_match_notes': 'ITIS valid match.',
                           'GAP_common_name': u'Sage Sparrow',
                           'GAP_scientific_name': 'Amphispiza belli',
                           'GAP_species_code': 'bSAGSx',
                           'GAP_taxa_notes': 'The Sage Sparrow (Artemisiospiza belli) has been the subject of much debate about its taxonomic status.  As of May 2013, the species is comprised of five subspecies which occur in the U.S. (A. b. belli, A. b. canescens,  A. b. cinerea, A. b. clementeae, A. b. nevadensis). A. b. nevadensis is likely to be elevated to spec

 u'Bubulcus ibis': {'GAP-ITIS_TSN': 174803,
                    'GAP-ITIS_match_notes': 'ITIS valid match.',
                    'GAP_common_name': u'Cattle Egret',
                    'GAP_scientific_name': 'Bubulcus ibis',
                    'GAP_species_code': 'bCAEGx',
                    'GAP_taxa_notes': nan,
                    'SGCN-GAP_match_basis': 'TSN',
                    'SGCN-GAP_match_notes': 'exact match',
                    'SGCN-ITIS_TSN': 174803,
                    'SGCN_common_name': u'Cattle Egret',
                    'SGCN_id': u'Bubulcus ibis'},
 u'Bucephala albeola': {'GAP-ITIS_TSN': 175145,
                        'GAP-ITIS_match_notes': 'ITIS valid match.',
                        'GAP_common_name': u'Bufflehead',
                        'GAP_scientific_name': 'Bucephala albeola',
                        'GAP_species_code': 'bBUFFx',
                        'GAP_taxa_notes': nan,
                        'SGCN-GAP_match_basis': 'TSN',
                     

...

                        'SGCN_common_name': u'Arctic Tern',
                        'SGCN_id': u'Sterna paradisaea'},
 u'Sternotherus carinatus': {'GAP-ITIS_TSN': 173759,
                             'GAP-ITIS_match_notes': 'ITIS valid match.',
                             'GAP_common_name': u'Razor-backed Musk Turtle',
                             'GAP_scientific_name': 'Sternotherus carinatus',
                             'GAP_species_code': 'rRBMTx',
                             'GAP_taxa_notes': nan,
                             'SGCN-GAP_match_basis': 'TSN',
                             'SGCN-GAP_match_notes': 'exact match',
                             'SGCN-ITIS_TSN': 173759,
                             'SGCN_common_name': u'Razor-backed Musk Turtle',
                             'SGCN_id': u'Sternotherus carinatus'},
 u'Sternotherus depressus': {'GAP-ITIS_TSN': 173760,
                             'GAP-ITIS_match_notes': 'ITIS valid match.',
                             'GAP_

                              'GAP_taxa_notes': nan,
                              'SGCN-GAP_match_basis': 'TSN',
                              'SGCN-GAP_match_notes': 'exact match',
                              'SGCN-ITIS_TSN': 180166,
                              'SGCN_common_name': u'Red Squirrel',
                              'SGCN_id': u'Tamiasciurus hudsonicus'},
 u'Tamiasciurus hudsonicus grahamensis': {'GAP-ITIS_TSN': 202353,
                                          'GAP-ITIS_match_notes': 'ITIS valid match.',
                                          'GAP_common_name': u'Mt. Graham Red Squirrel',
                                          'GAP_scientific_name': 'Tamiasciurus hudsonicus grahamensis',
                                          'GAP_species_code': 'mRESQg',
                                          'GAP_taxa_notes': nan,
                                          'SGCN-GAP_match_basis': 'TSN',
                                          'SGCN-GAP_match_notes': 'ex

 u'Toxostoma curvirostre': {'GAP-ITIS_TSN': 178637,
                            'GAP-ITIS_match_notes': 'ITIS valid match.',
                            'GAP_common_name': u'Curve-billed Thrasher',
                            'GAP_scientific_name': 'Toxostoma curvirostre',
                            'GAP_species_code': 'bCBTHx',
                            'GAP_taxa_notes': nan,
                            'SGCN-GAP_match_basis': 'TSN',
                            'SGCN-GAP_match_notes': 'exact match',
                            'SGCN-ITIS_TSN': 178637,
                            'SGCN_common_name': u'Curve-billed Thrasher',
                            'SGCN_id': u'Toxostoma curvirostre'},
 u'Toxostoma lecontei': {'GAP-ITIS_TSN': 178645,
                         'GAP-ITIS_match_notes': 'ITIS valid match.',
                         'GAP_common_name': u"Le Conte's Thrasher",
                         'GAP_scientific_name': 'Toxostoma lecontei',
                         'GAP_species_cod

                                     'SGCN-ITIS_TSN': 931054,
                                     'SGCN_common_name': u'Wyoming Ground Squirrel (Southwest Idaho popn.)',
                                     'SGCN_id': u'Urocitellus elegans nevadensis'},
 u'Urocitellus mollis': {'GAP-ITIS_TSN': 930320,
                         'GAP-ITIS_match_notes': 'ITIS valid match.',
                         'GAP_common_name': u'Piute Ground Squirrel',
                         'GAP_scientific_name': 'Urocitellus mollis',
                         'GAP_species_code': 'mPGSQx',
                         'GAP_taxa_notes': 'ITIS and NatureServe recognize the splitting of 8 genera (Notocitellus, Otospermophilus, Callospermophilus, Ictidomys, Poliocitellus, Xerospermophilus, and Urocitellus) from Spermophilus (which is now restricted to Eurasia) based on Helgen et al. (2009).\r\nSGW 29 September 2017',
                         'SGCN-GAP_match_basis': 'TSN',
                         'SGCN-GAP_match_notes': 

 u'Zapus princeps': {'GAP-ITIS_TSN': 180387,
                     'GAP-ITIS_match_notes': 'ITIS valid match.',
                     'GAP_common_name': u'Western Jumping Mouse',
                     'GAP_scientific_name': 'Zapus princeps',
                     'GAP_species_code': 'mWEMOx',
                     'GAP_taxa_notes': nan,
                     'SGCN-GAP_match_basis': 'TSN',
                     'SGCN-GAP_match_notes': 'exact match',
                     'SGCN-ITIS_TSN': 180387,
                     'SGCN_common_name': u'Western Jumping Mouse',
                     'SGCN_id': u'Zapus princeps'},
 u'Zapus trinotatus': {'GAP-ITIS_TSN': 180388,
                       'GAP-ITIS_match_notes': 'ITIS valid match.',
                       'GAP_common_name': u'Pacific Jumping Mouse',
                       'GAP_scientific_name': 'Zapus trinotatus',
                       'GAP_species_code': 'mPJMOx',
                       'GAP_taxa_notes': nan,
                       'SGCN-GAP_match_ba